项目

In [ ]:
数据集中有多个变量，包括客户的个人信息（如年龄、婚姻状况、工作等），以及营销过程中的一些详细信息（如通话时长、通话方式等）
用众数代替缺失值，从而排除了替换后过于极端的因子
星期的柱状图十分平均，对目标变量影响不大，排除该因子
age 变量存在许多异常值，于是对该变量进行了对数变换

In [ ]:
50%-25%-25% 拆分，对训练集进行 EDA
可视化，在相关性矩阵中发现有几个变量的相关性过高
对所有变量和目标变量进行卡方检验和fisher检验，排除了p>0.05的变量
对部分方法进行one-hot处理：
    kNN 通过计算样本之间的距离来进行分类
    LDA QDA
可以处理分类变量的方法：
    SVM
    决策树、随机森林
注意 one-hot 导致多出来很多维度

In [ ]:
kNN
进行了标准化处理。通过将所有数值型特征转换为均值为0、方差为1的标准正态分布，确保每个特征对距离计算的贡献是平等的。
通过交叉验证来选择最佳的 K 值，k-Fold Cross-Validation，分类准确率判定
分类完成后，通过混淆矩阵和其他评估指标来评估模型的表现。准确率、精确率、召回率和 F1 分数

In [ ]:
LDA
找到一个线性分离超平面，最大化类间距离，同时最小化类内距离
假设服从多元正态分布，假设所有类别共享同样的协方差矩阵
不满足上述条件因而改用 QDA
ROC AUC 都表明 QDA 更好

In [ ]:
SVM
当数据不能通过一个线性超平面分开时，SVM 通过核函数将数据映射到一个高维空间，在这个高维空间中找到一个线性可分的超平面
C（惩罚系数）：控制模型的松弛变量
非线性核函数没有显著提升性能，线性核计算效率更高

In [ ]:
决策树
基尼指数越低，表示数据更加纯净，分类效果更好
剪枝减少过拟合风险
预剪枝：限制树的最大深度、设置最小样本分裂数
后剪枝：交叉验证剪枝、复杂度惩罚

In [ ]:
Bagging
从原始数据集中随机有放回地抽取多个样本
在每个不同的训练子集上，训练一个独立的模型
对于分类问题，使用多数投票法

In [ ]:
Random Forest
内置的特征重要性评估机制
选择随机森林是基于它的高准确性、强大的特征选择能力，以及在避免过拟合方面的优异表现

In [ ]:
Q: 因子是如何选取的？

A:  1.初步筛选：用众数代替缺失值，从而排除了替换后过于极端的因子
    2.星期的柱状图十分平均，对目标变量影响不大，排除该因子
    3.相关性矩阵发现高度相关的变量，保留对目标影响更大的变量
    4.对所有变量和目标变量进行卡方检验和fisher检验，排除了p>0.05的变量
    5.通过交叉验证和超参数调优，最终选择随机森林作为最佳模型，确定了最终重要的特征

In [ ]:
Q: 我这个项目的目标是什么？是用来预测未来客户是否会订购？还是确定影响客户订购的因子？

A: 确定表现最好的模型以选定因子，从而在这个模型进行预测会有很好的表现

In [ ]:
Q: 模型确定以后，如何在工作中使用？比如10w块钱每次推广投递是10块，给你10w个用户画像，如何利用模型进行筛选投放？

A:  predict(rf_model, newdata = test_data, type = "prob")获得每个客户属于某个类别的概率
    为每个用户输出一个响应概率，这个概率表示该客户订阅定期存款的可能性
    择响应概率最高的 10,000 个用户进行投放
    测试组：随机森林
    对照组：常规投放方法投放
    收集两组用户的表现数据，对比转化率、点击率

In [ ]:
Q: 讲解决策树具体的原理操作过程
A: 决策树通过递归地分裂数据，每次选择最佳特征进行分裂，直到满足停止条件

In [ ]:
模型部署分出线上与线下两个环境

线下环境是单机环境，用以做一些探索性或者验证性的分析，包含了分析建模的每一个步骤：特征工程、模型选择、模型评价、参数调节等

线上环境则是生产环境，是把线下调整好参数的模型传至线上对新搜集的数据进行实时反馈。
其背后的原理是借助 get post 方法把特征字段传递给 web 服务器，服务器将会用封装好的模型预测并返回参数数值

In [ ]:
fiery
开启一个网络服务实例并设置ip地址、端口
启动服务并同时开启监听，同时加载训练好的储存在 model.rds 文件中的模型
响应 http 请求，编辑函数定义对访问路径为 predict 的请求的操作逻辑